<a href="https://colab.research.google.com/github/JH-KIM-82/Final-Team1/blob/main/%EA%B2%80%EC%A6%9D_%ED%94%8C%EB%A0%88%EC%9D%B4%ED%83%80%EC%9E%84%EB%B3%84_%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =======================================================
# 무료 게임 유저의 플레이타임 및 리뷰 신뢰도 분석
# - '무료로 받은 게임(received_for_free)' vs '구매한 게임'
# =======================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 한글 폰트 설정
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)
plt.style.use('fivethirtyeight')

# =======================================================
# 0. 데이터 불러오기
# =======================================================
# 파일을 직접 업로드하거나 경로를 수정해 주세요.
try:
    df = pd.read_csv('/content/weighted_score_above_08_250925-최종.csv')
    print("데이터셋을 성공적으로 불러왔습니다.")
    df.info()
except FileNotFoundError:
    print("오류: 'weighted_score_above_08_250925.csv' 파일을 찾을 수 없습니다.")
    exit()

# =======================================================
# 1. 데이터 전처리
# =======================================================
# 'received_for_free'와 'voted_up' 변수가 이미 전처리되어 있다고 가정
# 두 변수가 숫자(int) 타입인지 확인합니다.
print("\n'received_for_free' 변수 타입:", df['received_for_free'].dtype)
print("'voted_up' 변수 타입:", df['voted_up'].dtype)

# 'author_playtime_forever'를 시간 단위로 변환 (분 -> 시간)
df['author_playtime_hours'] = df['author_playtime_forever'] / 60

# 불필요한 열 제거 및 필요한 열만 선택
analysis_df = df[['received_for_free', 'author_playtime_hours', 'voted_up']].copy()

# =======================================================
# 2. 그룹별 데이터 분리
# =======================================================
free_reviews = analysis_df[analysis_df['received_for_free'] == 1].copy()
purchased_reviews = analysis_df[analysis_df['received_for_free'] == 0].copy()

print(f"\n무료로 받은 게임 리뷰 수: {len(free_reviews)}개")
print(f"구매한 게임 리뷰 수: {len(purchased_reviews)}개")

# =======================================================
# 3. 평균 플레이타임 비교
# =======================================================
free_playtime_mean = free_reviews['author_playtime_hours'].mean()
purchased_playtime_mean = purchased_reviews['author_playtime_hours'].mean()

print(f"\n무료 리뷰 그룹 평균 플레이타임: {free_playtime_mean:.2f} 시간")
print(f"구매 리뷰 그룹 평균 플레이타임: {purchased_playtime_mean:.2f} 시간")

# T-검정 (두 그룹 간 평균의 통계적 유의미성 검정)
t_stat, p_value = stats.ttest_ind(free_reviews['author_playtime_hours'], purchased_reviews['author_playtime_hours'], equal_var=False, nan_policy='omit')
print(f"플레이타임 T-검정 결과: t-statistic = {t_stat:.2f}, p-value = {p_value:.3e}")

# =======================================================
# 4. 리뷰 긍정률 비교
# =======================================================
free_voted_up_ratio = free_reviews['voted_up'].mean()
purchased_voted_up_ratio = purchased_reviews['voted_up'].mean()

print(f"\n무료 리뷰 긍정률: {free_voted_up_ratio:.2%}")
print(f"구매 리뷰 긍정률: {purchased_voted_up_ratio:.2%}")

# =======================================================
# 5. 시각화
# =======================================================
# 플레이타임 분포 시각화
plt.figure(figsize=(12, 6))
sns.histplot(free_reviews['author_playtime_hours'], bins=50, color='skyblue', label='무료 리뷰', kde=True, stat='density', alpha=0.6)
sns.histplot(purchased_reviews['author_playtime_hours'], bins=50, color='salmon', label='구매 리뷰', kde=True, stat='density', alpha=0.6)
plt.title('무료/구매 유저의 플레이타임 분포 비교')
plt.xlabel('플레이타임 (시간)')
plt.ylabel('밀도')
plt.xlim(0, 100) # 시각화를 위해 일부 구간만 표시
plt.legend()
plt.show()

# 리뷰 긍정률 막대 그래프 시각화
plt.figure(figsize=(8, 6))
sns.barplot(x=['무료 리뷰', '구매 리뷰'], y=[free_voted_up_ratio, purchased_voted_up_ratio], palette=['skyblue', 'salmon'])
plt.title('무료/구매 유저의 리뷰 긍정률 비교')
plt.ylabel('리뷰 긍정 비율')
plt.ylim(0.5, 1.0) # 0.5부터 시작하여 차이 강조
plt.show()
